<h1><center>Laboratorio 8: Aprendizaje Supervisado 🔮</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesor: Ignacio Meza y Gabriel Iturra
- Auxiliar: Sebastián Tinoco
- Ayudante: Arturo Lazcano y Angelo Muñoz

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Sebastián Ojeda G.


### **Link de repositorio de GitHub:** `https://github.com/sebajedi23/MDS7202`

## Temas a tratar

- Aprendizaje Supervisado
- Flujos de datos a través de `Pipelines`.

## Reglas

- **Grupos de 2 personas**
- Asistencia **obligatoria** a instrucciones del lab (viernes 16.15). Luego, pueden quedarse trabajando en las salas o irse.
- **No se revisarán entregas de personas ausentes**.
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.

## Objetivos principales del laboratorio

- Comprender el funcionamiento de clasificadores/regresores.
- Generar múltiples modelos predictivos.
- Comprender las ventajas de crear modelos en pipeline vs hacer las operaciones a mano.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre `DataFrames`.

### Importamos librerias utiles 😸

In [2]:
# Libreria Core del lab.
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split

# Pre-procesamiento
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import DecisionTreeClassifier

# Metricas de evaluación
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import cohen_kappa_score

#Libreria para plotear
!pip install --upgrade plotly
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.6/15.6 MB 55.1 MB/s eta 0:00:00
  Attempting uninstall: plotly
    Found existing installation: plotly 5.15.0
    Uninstalling plotly-5.15.0:
      Successfully uninstalled plotly-5.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
lida 0.0.10 requires fastapi, which is not installed.
lida 0.0.10 requires kaleido, which is not installed.
lida 0.0.10 requires python-multipart, which is not installed.
lida 0.0.10 requires uvicorn, which is not installed.


https://www.kaggle.com/antoinekrajnc/soccer-players-statistics

# 1. Predicciones Futboleras

<p align="center">
  <img src="https://drive.google.com/uc?export=view&id=1BnO4tyh3vM2P199Ec9s3JjngQ4qQ9seP"
" width="300">
</p>


Tras el trágico despido de la mítica mascota de Maipú, Renacín decide adentrarse como consultor en el mercado futbolero, el cuál (para variar...) está cargado en especulaciones.

Como su principal tarea será asesorar a los directivos de los clubes sobre cuál jugador comprar y cuál no, Renacín desea generar modelos predictivos que evaluén distintas características de los jugadores; todo con el fin de tomar decisiones concretas basadas en los datos.

Sin embargo, su condición de corporeo le impidió tomar la versión anterior de MDS7202, por lo que este motivo Renacín contrata a su equipo para lograr su objetivo final. Dado que aún tiene fuertes vínculos con la dirección de deportes de la municipalidad, el corporeo le entrega base de datos con las estadísticas de cada jugador para que su equipo empieze a trabajar ya con un dataset listo para ser usado.


**Los Datos**

Para este laboratorio deberán trabajar con los csv `statsplayers.csv` y `salarios.csv`, donde deberán aplicar algoritmos de de aprendizaje supervisado (clasificación y regresión) en base a características que describen de jugadores de futbol.

Para comenzar cargue el dataset señalado y a continuación vea el reporte **`Player_Stats_Report.html`** (adjunto en la carpeta del enunciado) que describe las características principales del `DataFrame`.

In [3]:
# Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
try:
    from google.colab import drive
    drive.mount("/content/drive")
    path = '/content/drive/MyDrive/Colab Notebooks/Laboratorio de programación científica para ciencia de datos/Lab 8/'
except:
    print('Ignorando conexión drive-colab')

Mounted at /content/drive


In [4]:
df_players = pd.read_csv(path + 'stats_players.csv')
df_players

,Name,Nationality,National_Position,Club_Position,Height,Weight,Preffered_Foot,Age,Work_Rate,Weak_foot,...,Agility,Jumping,Heading,Shot_Power,Finishing,Long_Shots,Curve,Freekick_Accuracy,Penalties,Volleys
0,Cristiano Ronaldo,Portugal,LS,LW,185,80,Right,32,High / Low,4,...,90,95,85,92,93,90,81,76,85,88
1,Lionel Messi,Argentina,RW,RW,170,72,Left,29,Medium / Medium,4,...,90,68,71,85,95,88,89,90,74,85
2,Neymar,Brazil,LW,LW,174,68,Right,25,High / Medium,5,...,96,61,62,78,89,77,79,84,81,83
3,Luis Suárez,Uruguay,LS,ST,182,85,Right,30,High / Medium,4,...,86,69,77,87,94,86,86,84,85,88
4,Manuel Neuer,Germany,GK,GK,193,92,Right,31,Medium / Medium,4,...,52,78,25,25,13,16,14,11,47,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17583,Adam Dunbar,Republic of Ireland,NaN,Sub,183,82,Right,19,Medium / Medium,1,...,27,56,14,16,13,13,11,13,15,12
17584,Dylan McGoey,Republic of Ireland,NaN,Sub,185,80,Right,19,Medium / Medium,2,...,28,53,12,17,12,11,12,13,16,12
17585,Tommy Ouldridge,England,NaN,Res,173,61,Right,18,High / Medium,2,...,54,61,41,44,28,42,35,36,42,37
17586,Mark Foden,Scotland,NaN,Sub,180,80,Right,21,Medium / Medium,3,...,34,48,15,23,14,12,13,12,24,12


## 1.1 Predicción de Seleccionados Nacionales

Como primera tarea, Renacín, intrigado por la posibilidad de saber qué tan reconocido es un jugador, le consulta a su equipo si es posible predecir si un jugador será o no seleccionado nacional a partir de sus estadísticas en el juego.

<p align="center">
  <img src="https://thumbs.gfycat.com/ThornyFrankFoxhound-size_restricted.gif" width="500">
</p>

### 1.1.1 Generación de Labels para la Clasificación [Sin Puntaje]


Primero comience generando las labels para la clasificación. Para esto, trabaje sobre el atributo ```National_Position``` suponiendo que los valores nulos son jugadores no seleccionados para representar a su país.

Hecho esto, ¿Cuantos ejemplos por cada clase se tienen? Comente lo que observa.



**Respuesta:**

In [5]:
#Creamos la columna National_Team_Selection con valor 1 para los nominados y 0 para los no nominados
df_players['National_Team_Selection'] = ~df_players.loc[:,'National_Position'].isna()*1
df_players['National_Team_Selection'].value_counts()

0    16513
1     1075
Name: National_Team_Selection, dtype: int64

Luego de creadas las clases se puede ver que la clase 0 (no seleccionados) tiene 16513 jugadores, mientras que la clase 1 (seleccionados) tiene 1075, por lo que es evidente que la mayoría de los jugadores de la base de datos no fueron convocados a sus respectivas selecciones nacionales, siendo su número más de 15 veces el número de jugadores que sí fueron seleccionados.

### 1.1.2 Camino a la clasificación [1 punto]

<p align="center">
  <img src="https://c.tenor.com/SoXZJHWnhKAAAAAC/pitbull-medel.gif" width="500">
</p>

Para preprocesar el dataset, genere un `ColumnTransformer` en donde especifique las transformaciones que hay que realizar para cada columna (por ejemplo StandarScaler, MinMaxScaler, OneHotEncoder, etc...) para que puedan ser utilizadas correctamente por el modelo predictivo y guardelo en algúna variable.

Luego, comente y justifique las transformaciones elegidas sobre cada una de las variables (para esto utilice el material ```Player_Stats_Report.html``` que viene en el zip del lab), al igual que las transformaciones aplicadas.

Hecho lo anterior, **defina al menos 3 pipelines para la clasificación**, en donde utilice el mismo ColumnTransformer definido anteriormente, pero que varie entre cada pipeline los clasificadores.

Para seleccionar los clasificadores más adecuados, utilice la siguiente guía:

<p align="center">
  <img src="https://scikit-learn.org/stable/_static/ml_map.png" width="500">
</p>

<br/>

Con ella, comente y justifique cada una de las decisiones tomadas al momento de desarrollar su pipeline.

**Nota:** Si tiene problemas al utilizar OneHotEncoder puede utilizar el parámetro handle_unknown='ignore'. Esto hace que en la codificación se omitan las categorias que no aparecen en el entrenamiento. Pregunta dudosa (no tiene puntaje), ¿esto tiene sentido a nivel de modelos?.

**To-Do:**

- [ ] Genere un ```ColumnTransformer``` enfocado en preprocesar los datos.
- [ ] Indicar y Justificar que preprocesamiento utiliza sobre cada columna.
- [ ] Crear 3 ``pipelines`` con diferentes clasificadores.
- [ ] Para seleccionar los clasificadores base sus decisiones en la siguiente [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html)
- [ ] No entrenar los ``pipelines`` aún.

**Nota:** No es necesario entrenar los clasificadores aún.



---


**Respuesta:**

---



Revisando las variables en el archivo html entregado es posible establecer los tipos de variables que se tienen, esto ya que se notó que estas siguen el formato del videojuego EA Sports FC (ex FIFA) que siguen la siguiente forma:



1.   Variables de texto: Se incluyen variables de texto que no influyen en el juego del jugador como sus nombres o países.
2.   Variables categóricas: Muestran categorías del tipo de juego del futbolista que no presentan algún orden jerárquico, por ejemplo el pie hábil, el nivel de esfuerzo o las posiciones en que juega.
3.   Variables numéricas: Variables que representan niveles de habilidad en cierto aspecto del juego en valores de 0 a 100 obtenibles mediante mediciones de las cualidades del jugador.
4.   Variables ordinales: Variables numéricas que representan niveles de habilidad en cierto aspecto del juego en valores de 0 a 5 estrellas. Debido a esta clasificación, el orden de ellas es importante (ya que 5 estrellas es mejor que 4), pero la distancia entre ellas no es necesariamente cuantificable.

In [21]:
#Creamos listas para almacenar las variables de cada tipo
text = ['Name','Nationality','National_Team_Selection','National_Position','Club_Position']

categorical = ['Preffered_Foot','Work_Rate']

numeric = ['Age','Weight','Height','Ball_Control', 'Dribbling', 'Marking', 'Sliding_Tackle',
       'Standing_Tackle', 'Aggression', 'Reactions', 'Interceptions', 'Vision',
       'Composure', 'Crossing', 'Short_Pass', 'Long_Pass', 'Acceleration',
       'Speed', 'Stamina', 'Strength', 'Balance', 'Agility', 'Jumping',
       'Heading', 'Shot_Power', 'Finishing', 'Long_Shots', 'Curve',
       'Freekick_Accuracy', 'Penalties', 'Volleys']

ordinal = ['Weak_foot','Skill_Moves']

In [22]:
from sklearn.preprocessing import OrdinalEncoder
ct = ColumnTransformer(
    [
        ('Scaler',MinMaxScaler(),numeric),
        ('One Hot',OneHotEncoder(sparse=False, handle_unknown="ignore",drop='if_binary'),categorical),
        ('Ordinal',OrdinalEncoder(),ordinal)
    ],
    remainder='passthrough'
)

Basado en los datos y la guía proporcionada se seleccionan los modelos a utilizar.

*   Siguiendo la guía se tiene que tenemos más de 50 muestras y buscamos predecir categorías a la vez que tenemos data etiquetada, con esto se llega a la implementación de un SVC lineal debido a que se tienen cerca de 18 mil filas, lejos de las 100 mil que se recomiendan para la utilización de un clasificador SGD.
*   Posterior a esto, dado que los datos no son de texto se procede con el siguiente paso sugerido que corresponde al modelo Nearest Neighbors.
*   Finalmente se llega a la opción de utilizar un clasificador de conjuntos, entre los que se selecciona Random Forest debido a la familiaridad que se tiene con este método de ensamblado de árboles.

In [23]:
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier

pip_svc = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model",SVC(kernel='linear',random_state=42) )
])

pip_knn = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model",KNeighborsClassifier() )
])

pip_rf = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model",RandomForestClassifier(random_state=42) )
])

### 1.1.3 Entrenemos los pipelines [1 punto]

<p align="center">
  <img src="https://i.gifer.com/7Cc8.gif" width="350">
</p>


Ahora, entrene los `pipeline` generados en los pasos anteriores. Para esto, primero separe los datos de entrenamiento en un conjunto de entrenamiento y de prueba  (la proporción queda a su juicio).

En este paso, seleccione los ejemplos de forma aleatoria e intente mantener la distribución original de labels de cada clase en los conjuntos de prueba/entrenamiento. (vea la documentación de `train_test_split`).


Luego, entrene los pipelines

Una vez entrenado su modelo, evalue su rendimiento a través de diferentes métricas, comentando que significa cada uno de los valores obtenidos. Puede usar la función `classification_report` para corroborar sus resultados.

- ¿Qué implican los valores de accuracy, precisión y recall de la clase positiva (la que indica que un jugador es seleccionado nacional)?
- ¿Podrían mejorarse los resultados?, ¿Cómo?
- ¿Influye la cantidad de ejemplos por clase?

**To-Do**:

- [ ] Separar el conjunto de datos en entrenamiento y de prueba.
- [ ] Entrenar los `pipelines`.
- [ ] Utilizar las métricas para evaluar los modelos generados.


**Respuesta:**

In [24]:
labels = df_players.loc[:, "National_Team_Selection"]
#Eliminamos las columnas de tipo texto
features = df_players.drop(columns=text)
#80% entrenamiento y 20% test
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.3,random_state=42,stratify=labels)

##Resultados con SVC

In [25]:
from sklearn.metrics import confusion_matrix, classification_report, f1_score
import prettytable as pt
import warnings

warnings.filterwarnings("ignore")

# Fit
pip_svc.fit(X_train, y_train)

# Predict
y_pred = pip_svc.predict(X_train)

# Matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Informe de clasificación
classification_rep = classification_report(y_train, y_pred, zero_division=1)

# F1-Score
f1 = f1_score(y_train, y_pred).round(2)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table.add_row(["Real Negativa", conf_matrix[0, 0], conf_matrix[0, 1]])
table.add_row(["Real Positiva", conf_matrix[1, 0], conf_matrix[1, 1]])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con svc:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con svc:")
print(classification_rep)

print(f"\nF1-Score en conjunto de entrenamiento con svc: {f1}")

Matriz de confusión en conjunto de entrenamiento con svc:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |        11559        |          0          |
| Real Positiva |         752         |          0          |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de entrenamiento con svc:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97     11559
           1       1.00      0.00      0.00       752

    accuracy                           0.94     12311
   macro avg       0.97      0.50      0.48     12311
weighted avg       0.94      0.94      0.91     12311


F1-Score en conjunto de entrenamiento con svc: 0.0


In [26]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

# Predict en conjunto de prueba
y_pred_test = pip_svc.predict(X_test)

# Matriz de confusión
conf_matrix_test = confusion_matrix(y_test, y_pred_test)

# Informe de clasificación
classification_rep_test = classification_report(y_test, y_pred_test, zero_division=1)

# F1-Score
f1_test = f1_score(y_test, y_pred_test).round(2)

# Crear una tabla para la matriz de confusión
table_test = pt.PrettyTable()
table_test.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table_test.add_row(["Real Negativa", conf_matrix_test[0, 0], conf_matrix_test[0, 1]])
table_test.add_row(["Real Positiva", conf_matrix_test[1, 0], conf_matrix_test[1, 1]])

# Imprimir resultados en conjunto de prueba
print("Matriz de confusión en conjunto de prueba con svc:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con svc:")
print(classification_rep_test)

print(f"\nF1-Score en conjunto de prueba con svc: {f1_test}")

Matriz de confusión en conjunto de prueba con svc:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |         4954        |          0          |
| Real Positiva |         323         |          0          |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de prueba con svc:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       1.00      0.00      0.00       323

    accuracy                           0.94      5277
   macro avg       0.97      0.50      0.48      5277
weighted avg       0.94      0.94      0.91      5277


F1-Score en conjunto de prueba con svc: 0.0


##Resultados con KNN

In [27]:
warnings.filterwarnings("ignore")

# Fit
pip_knn.fit(X_train, y_train)

# Predict
y_pred = pip_knn.predict(X_train)

# Matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Informe de clasificación
classification_rep = classification_report(y_train, y_pred, zero_division=1)

# F1-Score
f1 = f1_score(y_train, y_pred).round(2)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table.add_row(["Real Negativa", conf_matrix[0, 0], conf_matrix[0, 1]])
table.add_row(["Real Positiva", conf_matrix[1, 0], conf_matrix[1, 1]])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con knn:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con knn:")
print(classification_rep)

print(f"\nF1-Score en conjunto de entrenamiento con knn: {f1}")

Matriz de confusión en conjunto de entrenamiento con knn:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |        11524        |          35         |
| Real Positiva |         659         |          93         |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de entrenamiento con knn:
              precision    recall  f1-score   support

           0       0.95      1.00      0.97     11559
           1       0.73      0.12      0.21       752

    accuracy                           0.94     12311
   macro avg       0.84      0.56      0.59     12311
weighted avg       0.93      0.94      0.92     12311


F1-Score en conjunto de entrenamiento con knn: 0.21


In [28]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

# Predict en conjunto de prueba
y_pred_test = pip_knn.predict(X_test)

# Matriz de confusión
conf_matrix_test = confusion_matrix(y_test, y_pred_test)

# Informe de clasificación
classification_rep_test = classification_report(y_test, y_pred_test, zero_division=1)

# F1-Score
f1_test = f1_score(y_test, y_pred_test).round(2)

# Crear una tabla para la matriz de confusión
table_test = pt.PrettyTable()
table_test.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table_test.add_row(["Real Negativa", conf_matrix_test[0, 0], conf_matrix_test[0, 1]])
table_test.add_row(["Real Positiva", conf_matrix_test[1, 0], conf_matrix_test[1, 1]])

# Imprimir resultados en conjunto de prueba
print("Matriz de confusión en conjunto de prueba con knn:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con knn:")
print(classification_rep_test)

print(f"\nF1-Score en conjunto de prueba con knn: {f1_test}")


Matriz de confusión en conjunto de prueba con knn:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |         4929        |          25         |
| Real Positiva |         313         |          10         |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de prueba con knn:
              precision    recall  f1-score   support

           0       0.94      0.99      0.97      4954
           1       0.29      0.03      0.06       323

    accuracy                           0.94      5277
   macro avg       0.61      0.51      0.51      5277
weighted avg       0.90      0.94      0.91      5277


F1-Score en conjunto de prueba con knn: 0.06


##Resultados con RandomForest

In [29]:
warnings.filterwarnings("ignore")

# Fit
pip_rf.fit(X_train, y_train)

# Predict
y_pred = pip_rf.predict(X_train)

# Matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Informe de clasificación
classification_rep = classification_report(y_train, y_pred, zero_division=1)

# F1-Score
f1 = f1_score(y_train, y_pred).round(2)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table.add_row(["Real Negativa", conf_matrix[0, 0], conf_matrix[0, 1]])
table.add_row(["Real Positiva", conf_matrix[1, 0], conf_matrix[1, 1]])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con rf:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con rf:")
print(classification_rep)

print(f"\nF1-Score en conjunto de entrenamiento con rf: {f1}")


Matriz de confusión en conjunto de entrenamiento con rf:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |        11559        |          0          |
| Real Positiva |          0          |         752         |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de entrenamiento con rf:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     11559
           1       1.00      1.00      1.00       752

    accuracy                           1.00     12311
   macro avg       1.00      1.00      1.00     12311
weighted avg       1.00      1.00      1.00     12311


F1-Score en conjunto de entrenamiento con rf: 1.0


In [30]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

# Predict en conjunto de prueba
y_pred_test = pip_rf.predict(X_test)

# Matriz de confusión
conf_matrix_test = confusion_matrix(y_test, y_pred_test)

# Informe de clasificación
classification_rep_test = classification_report(y_test, y_pred_test, zero_division=1)

# F1-Score
f1_test = f1_score(y_test, y_pred_test).round(2)

# Crear una tabla para la matriz de confusión
table_test = pt.PrettyTable()
table_test.field_names = ["", "Predicción Negativa", "Predicción Positiva"]
table_test.add_row(["Real Negativa", conf_matrix_test[0, 0], conf_matrix_test[0, 1]])
table_test.add_row(["Real Positiva", conf_matrix_test[1, 0], conf_matrix_test[1, 1]])

# Imprimir resultados en conjunto de prueba
print("Matriz de confusión en conjunto de prueba con rf:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con rf:")
print(classification_rep_test)

print(f"\nF1-Score en conjunto de prueba con rf: {f1_test}")


Matriz de confusión en conjunto de prueba con rf:
+---------------+---------------------+---------------------+
|               | Predicción Negativa | Predicción Positiva |
+---------------+---------------------+---------------------+
| Real Negativa |         4938        |          16         |
| Real Positiva |         300         |          23         |
+---------------+---------------------+---------------------+

Informe de clasificación en conjunto de prueba con rf:
              precision    recall  f1-score   support

           0       0.94      1.00      0.97      4954
           1       0.59      0.07      0.13       323

    accuracy                           0.94      5277
   macro avg       0.77      0.53      0.55      5277
weighted avg       0.92      0.94      0.92      5277


F1-Score en conjunto de prueba con rf: 0.13


**Respuesta**:

```
Respecto a las métricas se puede indicar que el accuracy corresponde a la proporción de ejemplos clasificados correctamente en
comparación al total de ejemplos; Precision es la proporción de verdaderos positivos respecto a la totalidad de predicciones
positivas; Recall es la proporción de muestras clasificadas como positivas respecto a la totalidad de muestras positivas.

Estas métricas nos dan una idea de que tan bien se realiza la clasificación y hacia donde tiende a fallar más el modelo, por esto
mismo la métrica usada para comparar sus desempeños será el F1 score que combina tanto precision como recall, mostrándonos la
capacidad del modelo para clasificar correctamente ejemplos positivos y para clasificar todos los ejemplos positivos.


A partir de esto se tiene que el mejor modelo es Random Forest con un F1 score de 0.13, mientras que KNN tiene 0.06 y SVC 0.00


Los resultados actuales tienen margen de mejora, ya que los modelos se entrenaron con los parámetros por defecto y sin utilizar
técnicas como la validación cruzada ni una métrica de evaluación específica para optimizar el rendimiento. Además, se empleó un
conjunto de datos de entrenamiento desbalanceado, lo que condujo a que el modelo se enfocara en clasificar con mayor precisión los
ejemplos de la clase 0, es decir, aquellos que no fueron seleccionados, que corresponden a la clase mayoritaria.
```

## 1.2 Predicción de posiciones de jugadores [2 puntos]

<p align="center">
  <img src="https://pbs.twimg.com/media/E1rfA1aWEAYU6Ny.jpg" width="500">
</p>

En una nueva jornada de desmesuradas transacciones deportivas, Renacín escuchó a sus colegas discutir acerca de que el precio de cada jugador depende en gran medida de la posición en la cancha en la que juega. Y además, que hay bastantes jugadores nuevos que no tienen muy claro en que posición verdaderamente brillarían, por lo que actualmente puede que actualmente estén jugando en posiciones sub-optimas.

Viendo que los resultados del primer análisis no son tan esperanzadores, el corporeo los comanda a cambiar su tarea: ahora, les solicita que construyan un clasificador enfocado en predecir la mejor posición de los jugadores en la cancha según sus características.

Para lograr esto, primero, les pide que etiqueten de la siguiente manera los valores que aparecen en el atributo ```Club_Position```, pidiendo que agrupen los valores en los siguientes grupos:

**Nota**:  Renacín les recalca que **no deben utilizar los valores ```Sub``` y ```Res``` de esta columna**.

```python
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']
```

Cabe señalar que al igual como lo realizado con la clasificación binaria, deberá justificar en base a la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) la elección del clasificador y se deben comentar los resultados obtenidos en la clasificación.

**Nota:** Clasifique solamente con las clases señaladas, si observa mas clases eliminelas de la clasficación.

**To-Do**:

- [ ] Aplique las etiquetas descritas anteriormente en cada uno de los valores señalados en esta sección.
- [ ] Cuente cuantos por clase quedan.
- [ ] Entrene el nuevo pipeline y ejecute una evaluación de este.  
- [ ] Comente los resultados obtenidos.

**Respuesta:**

In [34]:
#Copiamos las listas dadas
ataque = ['ST', 'CF']
central_ataque = ['RW', 'CAM', 'LW']
central = ['RM', 'CM', 'LM']
central_defensa = ['RWB', 'CDM', 'LWB']
defensa = ['RB', 'CB', 'LB']
arquero = ['GK']

#Creamos un diccionario para almacenar los distintos roles que tiene cada posicion
Positions = {}

#Agregamos las listas al diccionario
Positions.update({k: 'ataque' for k in ataque})
Positions.update({k: 'central_ataque' for k in central_ataque})
Positions.update({k: 'central' for k in central})
Positions.update({k: 'central_defensa' for k in central_defensa})
Positions.update({k: 'defensa' for k in defensa})
Positions.update({k: 'arquero' for k in arquero})

In [35]:
Positions

{'ST': 'ataque',
 'CF': 'ataque',
 'RW': 'central_ataque',
 'CAM': 'central_ataque',
 'LW': 'central_ataque',
 'RM': 'central',
 'CM': 'central',
 'LM': 'central',
 'RWB': 'central_defensa',
 'CDM': 'central_defensa',
 'LWB': 'central_defensa',
 'RB': 'defensa',
 'CB': 'defensa',
 'LB': 'defensa',
 'GK': 'arquero'}

In [41]:
df_players = df_players.replace({'Club_Position': Positions})
df_filtered = df_players[(df_players["Club_Position"] != "Sub") & (df_players["Club_Position"] != "Res")]
df_filtered['Club_Position'].value_counts()

defensa            1180
central             907
RCB                 633
arquero             632
LCB                 631
central_ataque      581
ataque              430
LCM                 354
RCM                 353
LDM                 266
RDM                 266
central_defensa     209
RS                  206
LS                  205
LAM                  36
RAM                  36
LF                   12
RF                   12
Name: Club_Position, dtype: int64

In [42]:
#Eliminamos las clases que no fueron señaladas
#Creamos un filtro para identificar las filas con clases no señaladas
filtro = ~df_filtered['Club_Position'].isin(Positions.values())

#Eliminamos las filas que cumplan con el filtro
df_filtered = df_filtered[~filtro]

df_filtered['Club_Position'].value_counts()

defensa            1180
central             907
arquero             632
central_ataque      581
ataque              430
central_defensa     209
Name: Club_Position, dtype: int64

Se obtiene que la clase mayoritaria es defensa con 1180 entradas, seguida de central, arquero, central_ataque, ataque y por último la que menos muestras posee es central_defensa con 209.

Al igual que en el caso anterior, por el tipo de dato que se tiene y se busca predecir, se seleccionan los mismos tres modelos de la parte anterior (SVC lineal, KNN y Random Forest), por lo que se puede utilizar el mismo pipeline.

In [44]:
df_filtered.dropna(subset=["Club_Position"], inplace=True)
labels = df_filtered.loc[:, "Club_Position"]
features = df_filtered.drop(columns=text)
#80% entrenamiento y 20% test
X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size=0.2,random_state=42,stratify=labels)

##Resultados con SVC

In [52]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

pip_svc.fit(X_train, y_train)
y_pred = pip_svc.predict(X_train)

# Obtener los nombres reales de las clases
class_names = pip_svc.classes_

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Generar un informe de clasificación
classification_rep = classification_report(y_train, y_pred, target_names=class_names)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix):
    table.add_row([class_names[i], *row])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con svc:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con svc:")
print(classification_rep)

Matriz de confusión en conjunto de entrenamiento con svc:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   506   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  296   |    36   |       10       |        0        |    2    |
|     central     |    0    |   34   |   508   |       79       |        7        |    97   |
|  central_ataque |    0    |   32   |   255   |      158       |        2        |    18   |
| central_defensa |    0    |   0    |    23   |       1        |        41       |   102   |
|     defensa     |    0    |   0    |    23   |       0        |        6        |   915   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

I

In [53]:
# Predecir en el conjunto de prueba
y_pred = pip_svc.predict(X_test)

# Calcular la matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_pred, y_test)

# Generar un informe de clasificación en el conjunto de prueba
classification_rep_test = classification_report(y_pred, y_test, target_names=class_names)

# Crear una tabla para la matriz de confusión en el conjunto de prueba
table_test = pt.PrettyTable()
table_test.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix_test):
    table_test.add_row([class_names[i], *row])

# Imprimir resultados en el conjunto de prueba
print("Matriz de confusión en conjunto de prueba con svc:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con svc:")
print(classification_rep_test)


Matriz de confusión en conjunto de prueba con svc:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   126   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |   75   |    9    |       12       |        1        |    0    |
|     central     |    0    |   10   |   118   |       61       |        9        |    6    |
|  central_ataque |    0    |   1    |    27   |       39       |        2        |    0    |
| central_defensa |    0    |   0    |    3    |       0        |        5        |    2    |
|     defensa     |    0    |   0    |    25   |       4        |        25       |   228   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

Informe 

##Resultados con KNN

In [47]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

pip_knn.fit(X_train, y_train)
y_pred = pip_knn.predict(X_train)

# Obtener los nombres reales de las clases
class_names = pip_knn.classes_

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Generar un informe de clasificación
classification_rep = classification_report(y_train, y_pred, target_names=class_names)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix):
    table.add_row([class_names[i], *row])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con knn:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con knn:")
print(classification_rep)

Matriz de confusión en conjunto de entrenamiento con knn:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   506   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  283   |    44   |       14       |        2        |    1    |
|     central     |    0    |   43   |   528   |       65       |        9        |    80   |
|  central_ataque |    0    |   46   |   208   |      187       |        3        |    21   |
| central_defensa |    0    |   1    |    21   |       10       |        43       |    92   |
|     defensa     |    0    |   2    |    43   |       6        |        14       |   879   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

I

In [48]:
# Predecir en el conjunto de prueba
y_pred = pip_knn.predict(X_test)

# Calcular la matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_pred, y_test)

# Generar un informe de clasificación en el conjunto de prueba
classification_rep_test = classification_report(y_pred, y_test, target_names=class_names)

# Crear una tabla para la matriz de confusión en el conjunto de prueba
table_test = pt.PrettyTable()
table_test.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix_test):
    table_test.add_row([class_names[i], *row])

# Imprimir resultados en el conjunto de prueba
print("Matriz de confusión en conjunto de prueba con knn:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con knn:")
print(classification_rep_test)


Matriz de confusión en conjunto de prueba con knn:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   126   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |   57   |    13   |       19       |        0        |    1    |
|     central     |    0    |   21   |   101   |       70       |        11       |    10   |
|  central_ataque |    0    |   6    |    36   |       22       |        1        |    2    |
| central_defensa |    0    |   0    |    3    |       1        |        6        |    10   |
|     defensa     |    0    |   2    |    29   |       4        |        24       |   213   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

Informe 

##Resultados con RandomForest

In [49]:
# Suprimir advertencias de scikit-learn
warnings.filterwarnings("ignore")

pip_rf.fit(X_train, y_train)
y_pred = pip_rf.predict(X_train)

# Obtener los nombres reales de las clases
class_names = pip_rf.classes_

# Calcular la matriz de confusión
conf_matrix = confusion_matrix(y_train, y_pred)

# Generar un informe de clasificación
classification_rep = classification_report(y_train, y_pred, target_names=class_names)

# Crear una tabla para la matriz de confusión
table = pt.PrettyTable()
table.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix):
    table.add_row([class_names[i], *row])

# Imprimir resultados
print("Matriz de confusión en conjunto de entrenamiento con rf:")
print(table)

print("\nInforme de clasificación en conjunto de entrenamiento con rf:")
print(classification_rep)

Matriz de confusión en conjunto de entrenamiento con rf:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   506   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |  344   |    0    |       0        |        0        |    0    |
|     central     |    0    |   0    |   725   |       0        |        0        |    0    |
|  central_ataque |    0    |   0    |    0    |      465       |        0        |    0    |
| central_defensa |    0    |   0    |    0    |       0        |       167       |    0    |
|     defensa     |    0    |   0    |    0    |       0        |        0        |   944   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

In

In [50]:
# Predecir en el conjunto de prueba
y_pred = pip_rf.predict(X_test)

# Calcular la matriz de confusión en el conjunto de prueba
conf_matrix_test = confusion_matrix(y_pred, y_test)

# Generar un informe de clasificación en el conjunto de prueba
classification_rep_test = classification_report(y_pred, y_test, target_names=class_names)

# Crear una tabla para la matriz de confusión en el conjunto de prueba
table_test = pt.PrettyTable()
table_test.field_names = ["", *class_names]
for i, row in enumerate(conf_matrix_test):
    table_test.add_row([class_names[i], *row])

# Imprimir resultados en el conjunto de prueba
print("Matriz de confusión en conjunto de prueba con rf:")
print(table_test)

print("\nInforme de clasificación en conjunto de prueba con rf:")
print(classification_rep_test)


Matriz de confusión en conjunto de prueba con rf:
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|                 | arquero | ataque | central | central_ataque | central_defensa | defensa |
+-----------------+---------+--------+---------+----------------+-----------------+---------+
|     arquero     |   126   |   0    |    0    |       0        |        0        |    0    |
|      ataque     |    0    |   72   |    8    |       12       |        0        |    0    |
|     central     |    0    |   12   |   125   |       66       |        9        |    4    |
|  central_ataque |    0    |   2    |    25   |       33       |        4        |    2    |
| central_defensa |    0    |   0    |    2    |       2        |        5        |    3    |
|     defensa     |    0    |   0    |    22   |       3        |        24       |   227   |
+-----------------+---------+--------+---------+----------------+-----------------+---------+

Informe d

**Respuesta**:

```
Los resultados obtenidos para cada modelo son los siguientes:

SVC:
Accuracy=0.75
Precision=0.66
Recall=0.70
F1 score=0.65

KNN:
Accuracy=0.67
Precision=0.58
Recall=0.59
F1 score=0.57

RF:
Accuracy=0.75
Precision=0.65
Recall=0.68
F1 score=0.65


A la luz de estos resultados, se observa que los modelos SVC y Random Forest sobresalen como los mejores entre los tres modelos
evaluados.
Esto se debe a que presentan la mayor precisión, recall y F1-score promedio con diferencias ínfimas entre ellos. Por lo tanto, en
términos de rendimiento global, estos dos modelos son virtualmente igual de eficacez para esta tarea.
```

## 1.3 Predicción de Sueldos [2 puntos]

<p align="center">
  <img src="https://media.giphy.com/media/3oEjHWtHAFcOKS4iA0/giphy.gif" width="460">
</p>

Queriendo ahondar aún más en el mercado del balompíe, Renacin, logra obtener (de una manera no muy formal) los sueldos de múltiples futbolistas y los guarda en el archivo ```sueldos.csv```. Con ellos les solicita que generen un regresor que les permita predecir el sueldo de los futbolistas en base a las características de los pichichis, esto, debido a su motivación por invertir y/o realizar especulación sobre los sueldos de jugadores.

Renacin es claro señalando que deben seguir utilizando la [guía](https://scikit-learn.org/stable/tutorial/machine_learning_map/index.html) y comenten cada uno de los pasos realizados, para obtener su regresión lineal. Señalándoles que no aceptara un $R^2$ inferior a 0.35 para el modelo solicitado.

Para esta parte usted tiene total libertad en la generación del regresor, la unica exigencia es que utilice un pipeline para generar la regresión y utilice la metrica $R^2$ para medir el rendimiento de esta.

**To-Do**:

- [ ] Explique en que consiste la métrica $R^2$
- [ ] Generar un pipeline para la regresión.
- [ ] Obtener un regresor con un $R^2$ superior a  $0.35$.
- [ ] Comente sus resultados y si es posible mejorar los resultados obtenidos. ¿Se necesitarían más datos o otros tipos de características o una combinación de ambos?

**Respuesta**

In [68]:
df_salarios = pd.read_csv(path + 'salarios.csv')
df_salarios

,Unnamed: 0,Player,Weekly Salary
0,1528,Lionel Messi,1538905.0
1,1006,Cristiano Ronaldo,1248536.0
2,1529,Antoine Griezmann,997032.0
3,2004,Neymar,797726.0
4,1530,Gareth Bale,607706.0
...,...,...,...
2529,1527,Simone Ghidotti,283.0
2530,2539,Valentin Gendrey,100.0
2531,1001,John Yeboah,70.0
2532,1000,Antonis Aidonis,70.0


Primero, dado que este dataframe contiene menos filas que el anterior, hacemos un cruce de los datos que se encuentran en ambos.

In [83]:
df_players_salarios = df_salarios.merge(df_players, left_on='Player', right_on='Name')

In [84]:
# Contar valores nulos por columna
valores_nulos_por_columna = df_players_salarios.isnull().sum()

# Imprimir el recuento de valores nulos por columna
print(valores_nulos_por_columna)

Unnamed: 0                    0
Player                        0
Weekly Salary                 0
Name                          0
Nationality                   0
National_Position          1527
Club_Position                 0
Height                        0
Weight                        0
Preffered_Foot                0
Age                           0
Work_Rate                     0
Weak_foot                     0
Skill_Moves                   0
Ball_Control                  0
Dribbling                     0
Marking                       0
Sliding_Tackle                0
Standing_Tackle               0
Aggression                    0
Reactions                     0
Interceptions                 0
Vision                        0
Composure                     0
Crossing                      0
Short_Pass                    0
Long_Pass                     0
Acceleration                  0
Speed                         0
Stamina                       0
Strength                      0
Balance 

In [85]:
#Eliminamos la columna Player ya que es redundante con la columna Name
df_players_salarios.drop(columns=['Player'], inplace=True)

En primer lugar, ya que se tienen las mismas variables tipos de variables de antes, se mantienen las listas de tipos y ColumnTransformer anteriormente definidos.

Luego, de acuerdo al diagrama guía, vamos a la sección de regresión, y dado que tenemos menos de 100,000 observaciones y 40 características se utiliza Ridge Regression.

In [86]:
from sklearn.linear_model import Ridge

Ridge_reg = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model", Ridge())
])

In [87]:
from sklearn.metrics import r2_score

labels = df_players_salarios.loc[:, "Weekly Salary"]
features = df_players_salarios.drop(columns=text + ['Weekly Salary'])
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.2, random_state=42)

# Fit
Ridge_reg.fit(X_train, y_train)

# Predict
y_train_pred = Ridge_reg.predict(X_train)
y_test_pred = Ridge_reg.predict(X_test)

# Resultados
r2_train = r2_score(y_train, y_train_pred)
r2_test = r2_score(y_test, y_test_pred)

print("R^2 en el conjunto de entrenamiento: {:.2f}".format(r2_train))
print("R^2 en el conjunto de prueba: {:.2f}".format(r2_test))

R^2 en el conjunto de entrenamiento: 0.30
R^2 en el conjunto de prueba: 0.24


Dado que el valor de R^2 es muy bajo, se sigue con el diagrama guía, donde la siguiente opción es utilizar un Ensemble Regressor, por lo que utilizamos el regresor Random Forest.

In [88]:
from sklearn.ensemble import RandomForestRegressor
rf_reg = Pipeline(steps=[
                      ("preprocessing", ct),
                      ("model", RandomForestRegressor(random_state=42))
])

# Fit
rf_reg.fit(X_train, y_train)

# Predict
y_train_pred = rf_reg.predict(X_train)
y_test_pred = rf_reg.predict(X_test)

# Resultados para el modelo de Random Forest Regressor
r2_train_rf = r2_score(y_train, y_train_pred)
r2_test_rf = r2_score(y_test, y_test_pred)

print("R^2 en train (Random Forest Regressor): {:.2f}".format(r2_train_rf))
print("R^2 en test (Random Forest Regressor): {:.2f}".format(r2_test_rf))

R^2 en train (Random Forest Regressor): 0.96
R^2 en test (Random Forest Regressor): 0.57


**Respuesta**:

```
El modelo de regresión de Random Forest logra un R^2 de 0.57 en el conjunto de prueba, superando significativamente el puntaje de 0.24
obtenido por la regresión de Ridge. Esta superioridad se puede atribuir a la capacidad de los bosques aleatorios para modelar
relaciones no lineales y de alta dimensión entre las características. Por el contrario, la regresión Ridge es un modelo
lineal que asume una relación lineal entre las características y la variable objetivo. Si las relaciones subyacentes en los datos son
genuinamente no lineales, los modelos basados en árboles, como el bosque aleatorio, tienen más probabilidades de capturar estas
relaciones de manera efectiva en comparación con los modelos lineales, como Ridge.

Para mejorar aún más el rendimiento del modelo, se podría considerar la adición de más datos, ya que al cruzar los datos de
las bases de datos de salarios y jugadores quedan únicamente 1866 filas. Además, sería beneficioso aumentar la cantidad de
características relevantes para la regresión, como por ejemplo variables relacionadas a la economía de la liga o país en que juega.
```

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media2.giphy.com/media/l1J3CbFgn5o7DGRuE/giphy.gif">
</p>

<br>
<center>
<img src="https://i.kym-cdn.com/photos/images/original/001/194/195/b18.png" width=100 height=50 />
</center>
<br>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>